In [1]:
import pandas as pd
import cx_Oracle

In [82]:
ip = '172.16.80.74'
port = '1521'
SID = 'RPTDB'

dsn_tns = cx_Oracle.makedsn(ip, port, SID)
connection = cx_Oracle.connect('ANLYUSR', 'ANLYUSR$', dsn_tns)

In [181]:
tab_name = ['TBL_TRN_MB_CIB_ACCOUNT_SEGMENT',
'TBL_TRN_MB_CIB_ADDRESSLIST',
'TBL_TRN_MB_CIB_EMPLOYMENT_SEG',
'TBL_TRN_MB_CIB_RESPONSE_SCORE',
'TBL_TRN_MB_CIB_PHONELIST',
'TBL_TRN_MB_CIB_ID_SEGMENT',
'TBL_TRN_MB_CIB_ENQUIRYLIST',
'TBL_TRN_MB_CIB_NAMEHEADER',
'TBL_TRN_MB_CIB_ACCOUNTS_DETAIL',
'TBL_TRN_MB_HIGH_ACCOUNTSUMMARY',
'TBL_TRN_MB_HIGH_INDVRESPONSE',
'TBL_TRN_MB_HIGH_INDVRESP_ADDR',
'TBL_TRN_MB_HIGH_INDVRESP_ID',
'TBL_TRN_MB_HIGH_INDVRESP_PHONE',
'TBL_TRN_MB_HIGH_REQUEST',
'TBL_TRN_MB_HIGH_SCORE',
'TBL_TRN_CV_METADATA',
'TBL_TRN_CV_LOCATIONPROFILE',
'TBL_TRN_CV_CREDITCARDACCOUNT',
'TBL_TRN_CV_LOANACCOUNT',
'TBL_TRN_CV_LOANS',
'TBL_TRN_CV_CREDITSCORE_FRAUD',
'TBL_TRN_CV_BANKACCOUNT',
'TBL_TRN_CV_EMAILIDS',
'TBL_TRN_CV_INCOME',
'TBL_TRN_CV_ALTERNATECONTACT',
'TBL_TRN_CV_ACCOUNTS',
'TBL_TRN_CV_UTILITYACCOUNT',
'TBL_TRN_CV_CREDITCARD',
'TBL_TRN_MST_CV_DETAILS']

In [184]:
flag1 = True
for i in tab_name:
    
    query = "SELECT COLUMN_NAME FROM all_tab_columns \
    where OWNER = 'BIUSR' AND TABLE_NAME = {} ".format("'"+i+"'")
    
    columns_DF = pd.read_sql(query, con=connection)

    flag2 = True
    
    if '_CV_' in i: 
        for j in columns_DF.COLUMN_NAME.values:
        
            query1 = "SELECT tb1.status as Identifiers,count(tb1.status) as NULL_count from \
            (SELECT DECODE({},'null','is_null',null,'is_null','not_null')  as status FROM BIUSR.{} WHERE PROPNO like '%CD%') tb1 \
            group by tb1.status".format(j,i)
    
            result = pd.read_sql(query1, con=connection)
    
            result['Column_Name'] = j
            result['Table_Name'] = i
    
            if flag2:
                final_result = result
                flag2 = False
            else:
                final_result = pd.concat([final_result,result])
    else:
        for k in columns_DF.COLUMN_NAME.values:
        
            query1 = "SELECT tb1.status as Identifiers,count(tb1.status) as NULL_count from \
            (SELECT DECODE({},'null','is_null',null,'is_null','not_null')  as status FROM BIUSR.{} WHERE PROSPECT_NO like '%CD%') tb1 \
            group by tb1.status".format(k,i)
    
            result = pd.read_sql(query1, con=connection)
    
            result['Column_Name'] = k
            result['Table_Name'] = i
    
            if flag2:
                final_result = result
                flag2 = False
            else:
                final_result = pd.concat([final_result,result])
        
    if flag1:
        consol_list = final_result
        flag1 = False
    else:
        consol_list = pd.concat([consol_list,final_result])

In [189]:
consol_list

,IDENTIFIERS,NULL_COUNT,Column_Name,Table_Name
0,not_null,1613661,PROSPECT_NO,TBL_TRN_MB_CIB_ACCOUNT_SEGMENT
0,not_null,1613661,SEQ_ACCOUNT_NO,TBL_TRN_MB_CIB_ACCOUNT_SEGMENT
0,not_null,1613661,REPORTING_MEMBER_SHORTNAME,TBL_TRN_MB_CIB_ACCOUNT_SEGMENT
0,is_null,1613661,ACCOUNT_NUMBER,TBL_TRN_MB_CIB_ACCOUNT_SEGMENT
0,not_null,1613661,ACCOUNT_TYPE,TBL_TRN_MB_CIB_ACCOUNT_SEGMENT
0,not_null,1613661,OWNERSHIP_INDICATOR,TBL_TRN_MB_CIB_ACCOUNT_SEGMENT
0,is_null,4572,DATE_OPEN_DISBURSED,TBL_TRN_MB_CIB_ACCOUNT_SEGMENT
1,not_null,1609089,DATE_OPEN_DISBURSED,TBL_TRN_MB_CIB_ACCOUNT_SEGMENT
0,is_null,398986,DATE_LASTPAYMENT,TBL_TRN_MB_CIB_ACCOUNT_SEGMENT
1,not_null,1214675,DATE_LASTPAYMENT,TBL_TRN_MB_CIB_ACCOUNT_SEGMENT


In [197]:
dataset = consol_list

In [198]:
dataset['NULL_COUNT'] = dataset.NULL_COUNT.astype(int)

In [201]:
data_test = pd.pivot_table(dataset, values = 'NULL_COUNT',
               index=['Table_Name','Column_Name'],
               columns = 'IDENTIFIERS').reset_index().fillna(0)

In [202]:
data_test = pd.DataFrame(data_test.to_records())[['Table_Name','Column_Name','is_null','not_null']]

In [203]:
data_test['Total_Records'] = data_test['is_null']+data_test['not_null']

In [204]:
def cal_null_perc(row):
    try:
        return round((row['is_null']/row['Total_Records'])*100,2)
    except:
        return 0.0

In [205]:
data_test['null_percentage'] = data_test.apply(cal_null_perc, axis = 1)

In [206]:
data_test

,Table_Name,Column_Name,is_null,not_null,Total_Records,null_percentage
0,TBL_TRN_CV_ACCOUNTS,ACCOUNT_CNT,0.0,187323.0,187323.0,0.00
1,TBL_TRN_CV_ACCOUNTS,AMBMONTH0,0.0,187323.0,187323.0,0.00
2,TBL_TRN_CV_ACCOUNTS,AMBMONTH1,0.0,187323.0,187323.0,0.00
3,TBL_TRN_CV_ACCOUNTS,AMBMONTH2,0.0,187323.0,187323.0,0.00
4,TBL_TRN_CV_ACCOUNTS,BANKNAME,0.0,187323.0,187323.0,0.00
5,TBL_TRN_CV_ACCOUNTS,CREATED_DATE,0.0,187323.0,187323.0,0.00
6,TBL_TRN_CV_ACCOUNTS,CREDITMONTH0,0.0,187323.0,187323.0,0.00
7,TBL_TRN_CV_ACCOUNTS,CREDITMONTH1,0.0,187323.0,187323.0,0.00
8,TBL_TRN_CV_ACCOUNTS,CREDITMONTH2,0.0,187323.0,187323.0,0.00
9,TBL_TRN_CV_ACCOUNTS,DEBITMONTH0,0.0,187323.0,187323.0,0.00


In [207]:
data_test.to_csv("D:\CD\CD_MB_CIBIL_CRIF_CV_AUdit.csv")